# Table of Contents
1  Image概述
2  Image operator(mask, clip, select, addBands, export)
3  ImageCollection概述
4  ImageCollection operator(filter, select, set/get, calculation, toList)
5  Example 1. Calculate NDVI for Jiangxi and Extract NDWI for poyang Lake between 1982 to 2021;
6  Example 2. Download Image or ImageCollection from google earth engine;
7  Example 3. Extract NDVI values of climate stations from 1982 to 2021;
8  Example 4. Supervised Classification
9  Example 5. Unsupervised Classification (clustering) 

In [ ]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map

In [13]:
# -------------------------------------------------------
# 2  Image operator(mask, clip, select, addBands, export)
#     a.Image数据的基础操作
# -------------------------------------------------------
# 根据常量创建ee对象
image1 = ee.Image(1)
print(image1.getInfo())

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [14]:
# ee对象的合并
image2 = ee.Image(2)
image3 = ee.Image(3)
image4 = ee.Image.cat([image1, image2, image3])
print(image4.getInfo())



{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [6]:
# 向ee对象中添加波段i
image5 = image4.addBands(ee.Image(42))
print(image5.getInfo())


{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 42, 'max': 42}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [8]:
multiband = ee.Image([1, 2, 3])
print(multiband.getInfo())

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [9]:
# ee对象修改波段名 select的用法   重点！！！！！！！！！！！
renamed = multiband.select(
    [0, 1, 2], # old names
    ['band1', 'band2', 'band3']               # new names
)
print(renamed.getInfo())


{'type': 'Image', 'bands': [{'id': 'band1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'band2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'band3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [ ]:
# 导入ee图层中的指定波段————select函数，select有两种传递的参数方式
# 第一种直接传递波段索引值或波段名称或其名称的正则表达式组成的非列表序列，如下：
loadedImage = ee.Image('JAXA/ALOS/AW3D30/V2_2').select('.*?DSM', 'AVE_STK', 2)
Map.addLayer(loadedImage, {}, 'loadedImage')

In [ ]:
# 第二种传递两个列表，第一个列表用于波段筛选，第二个列表用于指定波段的新名称
# 两个列表的元素个数必须一致，如下：
loadedImage_All = ee.Image('JAXA/ALOS/AW3D30/V2_2').select([0, 1], ['A', 'B'])
Map.addLayer(loadedImage_All, {}, 'All')
Map.addLayer(loadedImage_All.select(0), {}, 'A')
Map.addLayer(loadedImage_All.select('B'), {}, 'B')

In [6]:
# props = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').propertyNames().getInfo()
print(properties)

['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'CPF_NAME', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'google:registration_count', 'PANCHROMATIC_SAMPLES', 'PANCHROMATIC_LINES', 'GEOMETRIC_RMSE_MODEL_Y', 'REFLECTIVE_LINES', 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE', 'GEOMETRIC_RMSE_MODEL_X', 'system:asset_size', 'system:

In [2]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
bandNames = image.bandNames().getInfo()
b1proj = image.select('B3').projection().getInfo()
b1scale = image.select('B4').projection().nominalScale().getInfo()
properties = image.propertyNames().getInfo()
cloudiness = image.get('CLOUD_COVER').getInfo()
date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss').getInfo()

In [9]:
ID = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').get('system:time_start').getInfo()
print(ID)

1395168392050


In [5]:
print(b1scale)

30


In [7]:
ID = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').get('system:id').getInfo()
print(ID)

LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318


In [11]:
print('CLOUD_COVER: ', cloudiness)

CLOUD_COVER:  0.05999999865889549


In [10]:
print('time_start: ', date)

time_start:  2014-03-18 18:46:32


In [20]:
Map

Map(bottom=405605.0, center=[37.7726, -122.3578], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [2]:
# // Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')

# // Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# // Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# # // Mask and display the binary layer.
Map.setCenter(-122.3578, 37.7726, 12)
Map.setOptions('SATELLITE')
Map.addLayer(bare.updateMask(bare), {}, 'bare')

In [15]:
img = image2.add(image3)
print(img.getInfo())

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 5, 'max': 5}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [3]:
# ------------------------------------------------------------------------------------------
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# ------------------------------------------------------------------------------------------
# // Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# // Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# // Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {min: 0, max: 3, 'palette': palette}, 'development zones')